In [87]:
import pandas as pd
import re
from nltk import bigrams
from nltk.lm.preprocessing import pad_both_ends
from sklearn.model_selection import train_test_split
from nltk.lm.preprocessing import padded_everygram_pipeline
from nltk.lm import MLE,Laplace
from nltk.lm import NgramCounter
import nltk

In [ ]:
nltk.download("all")

In [2]:
dados_portugues = pd.read_csv("/Users/gustavobrandao/Downloads/stackoverflow_portugues.csv")
dados_portugues.head()

,Id,Título,Questão,Tags,Pontuação,Visualizações
0,2402,Como fazer hash de senhas de forma segura?,"<p>Se eu fizer o <em><a href=""http://pt.wikipe...",<hash><segurança><senhas><criptografia>,350,22367
1,6441,Qual é a diferença entre INNER JOIN e OUTER JOIN?,<p>Qual é a diferença entre <code>INNER JOIN</...,<sql><join>,276,176953
2,579,Por que não devemos usar funções do tipo mysql_*?,<p>Uma dúvida muito comum é por que devemos pa...,<php><mysql>,226,9761
3,2539,As mensagens de erro devem se desculpar?,<p>É comum encontrar uma mensagem de erro que ...,<aplicação-web><gui><console><ux>,214,5075
4,17501,"Qual é a diferença de API, biblioteca e Framew...",<p>Me parecem termos muito próximos e eventual...,<api><framework><terminologia><biblioteca>,193,54191


In [3]:
dados_ingles = pd.read_csv("/Users/gustavobrandao/Downloads/stackoverflow_ingles.csv")
dados_ingles.head()

,Id,Título,Questão,Tags,Pontuação,Visualizações
0,11227809,Why is it faster to process a sorted array tha...,<p>Here is a piece of C++ code that seems very...,<java><c++><performance><optimization><branch-...,23057,1358574
1,927358,How do I undo the most recent local commits in...,<p>I accidentally committed the wrong files to...,<git><version-control><git-commit><undo>,19640,7906137
2,2003505,How do I delete a Git branch locally and remot...,<p>I want to delete a branch both locally and ...,<git><git-branch><git-remote>,15249,6940906
3,292357,What is the difference between 'git pull' and ...,<blockquote>\n <p><strong>Moderator Note:</st...,<git><git-pull><git-fetch>,11008,2543052
4,477816,What is the correct JSON content type?,"<p>I've been messing around with <a href=""http...",<json><http-headers><content-type>,9701,2478940


In [4]:
dados_espanhol = pd.read_csv("/Users/gustavobrandao/Downloads/stackoverflow_espanhol.csv")
dados_espanhol.head()

,Id,Título,Questão,Tags,Pontuação,Visualizações
0,18232,¿Cómo evitar la inyección SQL en PHP?,<p>Las sentencias dinámicas son sentencias SQL...,<php><mysql><sql><seguridad><inyección-sql>,169,38614
1,197,¿Por qué mis programas no pueden hacer cálculo...,<p>Unas veces los cálculos funcionan correctam...,<matemáticas><coma-flotante><lenguaje-agnóstico>,141,3860
2,36,¿Cuál es la diferencia entre un inner y un out...,<p>¿Cuál es la diferencia entre un <code>inner...,<mysql><sql><join>,97,53627
3,29177,¿Por qué es considerado una mala práctica util...,"<p>La mayoría de nosotros decimos, (muchas vec...",<variables><variables-globales><patrones-de-di...,89,9987
4,142,Validar un email en JavaScript que acepte todo...,<h3>Pregunta</h3>\n\n<p>¿Cómo validar un e-mai...,<javascript><validación><email><regex>,87,73129


In [5]:
#Usando Regex para encontrar as tags html nos textos
questao_portugues = dados_portugues.Questão[0]
tags = re.findall("<.*?>",questao_portugues)
tags

['<p>',
 '<em>',
 '<a href="http://pt.wikipedia.org/wiki/Fun%C3%A7%C3%A3o_de_embaralhamento_criptogr%C3%A1fico" rel="noreferrer">',
 '</a>',
 '</em>',
 '</p>',
 '<p>',
 '<a href="http://pt.wikipedia.org/wiki/Ataque_de_for%C3%A7a_bruta" rel="noreferrer">',
 '</a>',
 '<em>',
 '<a href="http://pt.wikipedia.org/wiki/Keylogger" rel="noreferrer">',
 '</a>',
 '</em>',
 '<a href="http://pt.wikipedia.org/wiki/Criptoan%C3%A1lise_de_mangueira_de_borracha" rel="noreferrer">',
 '<em>',
 '</em>',
 '</a>',
 '<em>',
 '</em>',
 '</p>',
 '<p>',
 '</p>',
 '<p>',
 '<em>',
 '</em>',
 '</p>']

In [6]:
#Tratando a frase
questao_portugues_v2 = re.sub("<.*?>","",questao_portugues)
questao_portugues_v3 = re.sub(r"\n","",questao_portugues_v2)
questao_portugues_v3

'Se eu fizer o hash de senhas antes de armazená-las em meu banco de dados é suficiente para evitar que elas sejam recuperadas por alguém?Estou falando apenas da recuperação diretamente do banco de dados e não qualquer outro tipo de ataque, como força bruta na página de login da aplicação, keylogger no cliente e criptoanálise rubberhose. Qualquer forma de hash não vai impedir esses ataques.Tenho preocupação em dificultar ou até impossibilitar a obtenção das senhas originais caso o banco de dados seja comprometido. Como dar maior garantia de segurança neste aspecto?Quais preocupações adicionais evitariam o acesso às senhas? Existem formas melhores de fazer esse hash?'

In [7]:
regex = re.compile(r"<p>")
regex.search(questao_portugues)

<re.Match object; span=(0, 3), match='<p>'>

In [8]:
def remove_texto(textos,regex):
    if type(textos) == str:
        texto_sem_tag = regex.sub("",textos)
        return texto_sem_tag
    else:
        return [regex.sub("",texto) for texto in textos]
regex = re.compile("<.*?>")
print(remove_texto(questao_portugues,regex))

Se eu fizer o hash de senhas antes de armazená-las em meu banco de dados é suficiente para evitar que elas sejam recuperadas por alguém?

Estou falando apenas da recuperação diretamente do banco de dados e não qualquer outro tipo de ataque, como força bruta na página de login da aplicação, keylogger no cliente e criptoanálise rubberhose. Qualquer forma de hash não vai impedir esses ataques.

Tenho preocupação em dificultar ou até impossibilitar a obtenção das senhas originais caso o banco de dados seja comprometido. Como dar maior garantia de segurança neste aspecto?

Quais preocupações adicionais evitariam o acesso às senhas? Existem formas melhores de fazer esse hash?



In [9]:
questao_ingles = dados_ingles.Questão[0]
questao_ingles

'<p>Here is a piece of C++ code that seems very peculiar. For some strange reason, sorting the data miraculously makes the code almost six times faster.</p>\n\n<pre class="lang-cpp prettyprint-override"><code>#include &lt;algorithm&gt;\n#include &lt;ctime&gt;\n#include &lt;iostream&gt;\n\nint main()\n{\n    // Generate data\n    const unsigned arraySize = 32768;\n    int data[arraySize];\n\n    for (unsigned c = 0; c &lt; arraySize; ++c)\n        data[c] = std::rand() % 256;\n\n    // !!! With this, the next loop runs faster\n    std::sort(data, data + arraySize);\n\n    // Test\n    clock_t start = clock();\n    long long sum = 0;\n\n    for (unsigned i = 0; i &lt; 100000; ++i)\n    {\n        // Primary loop\n        for (unsigned c = 0; c &lt; arraySize; ++c)\n        {\n            if (data[c] &gt;= 128)\n                sum += data[c];\n        }\n    }\n\n    double elapsedTime = static_cast&lt;double&gt;(clock() - start) / CLOCKS_PER_SEC;\n\n    std::cout &lt;&lt; elapsedTime &l

In [10]:
#Removendo os códigos com substituição por regex
regex = regex = re.compile("<code>(.|(\n))*?</code>")
print(remove_texto(questao_ingles,regex))

<p>Here is a piece of C++ code that seems very peculiar. For some strange reason, sorting the data miraculously makes the code almost six times faster.</p>

<pre class="lang-cpp prettyprint-override"></pre>

<ul>
<li>Without , the code runs in 11.54 seconds.</li>
<li>With the sorted data, the code runs in 1.93 seconds.</li>
</ul>

<p>Initially, I thought this might be just a language or compiler anomaly. So I tried it in Java.</p>

<pre class="lang-java prettyprint-override"></pre>

<p>With a somewhat similar but less extreme result.</p>

<hr>

<p>My first thought was that sorting brings the data into the cache, but then I thought how silly that is because the array was just generated.</p>

<ul>
<li>What is going on?</li>
<li>Why is it faster to process a sorted array than an unsorted array?</li>
<li>The code is summing up some independent terms, and the order should not matter.</li>
</ul>



In [11]:
questao_portugues

'<p>Se eu fizer o <em><a href="http://pt.wikipedia.org/wiki/Fun%C3%A7%C3%A3o_de_embaralhamento_criptogr%C3%A1fico" rel="noreferrer">hash</a></em> de senhas antes de armazená-las em meu banco de dados é suficiente para evitar que elas sejam recuperadas por alguém?</p>\n\n<p>Estou falando apenas da recuperação diretamente do banco de dados e não qualquer outro tipo de ataque, como <a href="http://pt.wikipedia.org/wiki/Ataque_de_for%C3%A7a_bruta" rel="noreferrer">força bruta</a> na página de login da aplicação, <em><a href="http://pt.wikipedia.org/wiki/Keylogger" rel="noreferrer">keylogger</a></em> no cliente e <a href="http://pt.wikipedia.org/wiki/Criptoan%C3%A1lise_de_mangueira_de_borracha" rel="noreferrer">criptoanálise <em>rubberhose</em></a>. Qualquer forma de <em>hash</em> não vai impedir esses ataques.</p>\n\n<p>Tenho preocupação em dificultar ou até impossibilitar a obtenção das senhas originais caso o banco de dados seja comprometido. Como dar maior garantia de segurança neste as

In [12]:
#Retirando trechos de código e tags html para questões em português
regex_nocode = re.compile("<code>(.|(\n))*?</code>")
dados_pt_nocode = [remove_texto(texto,regex_nocode) for texto in dados_portugues.Questão]
regex_html = re.compile("<.*?>")
dados_portugues["Pos_tratamento"] = [remove_texto(texto,regex_html) for texto in dados_pt_nocode]
dados_portugues.head()

,Id,Título,Questão,Tags,Pontuação,Visualizações,Pos_tratamento
0,2402,Como fazer hash de senhas de forma segura?,"<p>Se eu fizer o <em><a href=""http://pt.wikipe...",<hash><segurança><senhas><criptografia>,350,22367,Se eu fizer o hash de senhas antes de armazená...
1,6441,Qual é a diferença entre INNER JOIN e OUTER JOIN?,<p>Qual é a diferença entre <code>INNER JOIN</...,<sql><join>,276,176953,Qual é a diferença entre e ? Podem me dar alg...
2,579,Por que não devemos usar funções do tipo mysql_*?,<p>Uma dúvida muito comum é por que devemos pa...,<php><mysql>,226,9761,Uma dúvida muito comum é por que devemos parar...
3,2539,As mensagens de erro devem se desculpar?,<p>É comum encontrar uma mensagem de erro que ...,<aplicação-web><gui><console><ux>,214,5075,É comum encontrar uma mensagem de erro que diz...
4,17501,"Qual é a diferença de API, biblioteca e Framew...",<p>Me parecem termos muito próximos e eventual...,<api><framework><terminologia><biblioteca>,193,54191,Me parecem termos muito próximos e eventualmen...


In [13]:
#Retirando trechos de código e tags html para questões em espanhol
regex_nocode = re.compile("<code>(.|(\n))*?</code>")
dados_es_nocode = [remove_texto(texto,regex_nocode) for texto in dados_espanhol.Questão]
regex_html = re.compile("<.*?>")
dados_espanhol["Pos_tratamento"] = [remove_texto(texto,regex_html) for texto in dados_es_nocode]
dados_espanhol.head()

,Id,Título,Questão,Tags,Pontuação,Visualizações,Pos_tratamento
0,18232,¿Cómo evitar la inyección SQL en PHP?,<p>Las sentencias dinámicas son sentencias SQL...,<php><mysql><sql><seguridad><inyección-sql>,169,38614,Las sentencias dinámicas son sentencias SQL qu...
1,197,¿Por qué mis programas no pueden hacer cálculo...,<p>Unas veces los cálculos funcionan correctam...,<matemáticas><coma-flotante><lenguaje-agnóstico>,141,3860,Unas veces los cálculos funcionan correctament...
2,36,¿Cuál es la diferencia entre un inner y un out...,<p>¿Cuál es la diferencia entre un <code>inner...,<mysql><sql><join>,97,53627,¿Cuál es la diferencia entre un y un ?\n\nY ¿...
3,29177,¿Por qué es considerado una mala práctica util...,"<p>La mayoría de nosotros decimos, (muchas vec...",<variables><variables-globales><patrones-de-di...,89,9987,"La mayoría de nosotros decimos, (muchas veces ..."
4,142,Validar un email en JavaScript que acepte todo...,<h3>Pregunta</h3>\n\n<p>¿Cómo validar un e-mai...,<javascript><validación><email><regex>,87,73129,Pregunta\n\n¿Cómo validar un e-mail que acepte...


In [14]:
#Retirando trechos de código e tags html para questões em inglês
regex_nocode = re.compile("<code>(.|(\n))*?</code>")
dados_ing_nocode = [remove_texto(texto,regex_nocode) for texto in dados_ingles.Questão]
regex_html = re.compile("<.*?>")
dados_ingles["Pos_tratamento"] = [remove_texto(texto,regex_html) for texto in dados_ing_nocode]
dados_ingles.head()

,Id,Título,Questão,Tags,Pontuação,Visualizações,Pos_tratamento
0,11227809,Why is it faster to process a sorted array tha...,<p>Here is a piece of C++ code that seems very...,<java><c++><performance><optimization><branch-...,23057,1358574,Here is a piece of C++ code that seems very pe...
1,927358,How do I undo the most recent local commits in...,<p>I accidentally committed the wrong files to...,<git><version-control><git-commit><undo>,19640,7906137,I accidentally committed the wrong files to Gi...
2,2003505,How do I delete a Git branch locally and remot...,<p>I want to delete a branch both locally and ...,<git><git-branch><git-remote>,15249,6940906,I want to delete a branch both locally and rem...
3,292357,What is the difference between 'git pull' and ...,<blockquote>\n <p><strong>Moderator Note:</st...,<git><git-pull><git-fetch>,11008,2543052,\n Moderator Note: Given that this question h...
4,477816,What is the correct JSON content type?,"<p>I've been messing around with <a href=""http...",<json><http-headers><content-type>,9701,2478940,I've been messing around with JSON for some ti...


In [15]:
#Retirando pontuação 
regex_pontuacao = re.compile("[^\w\s]")
dados_es_tratado = regex_pontuacao.sub("",dados_espanhol.Pos_tratamento[0])

In [18]:
#Definindo função para deixar transformar as palavras em minúsculo e assim padroniza-las e melhorar a performance do modelo
def minusculo(textos):
    if type(textos) == str:
        return textos.lower()
    else:
        return [texto.lower() for texto in textos]
print(minusculo(dados_es_tratado))

las sentencias dinámicas son sentencias sql que se crean como cadenas de texto strings y en las que se insertanconcatenan valores obtenidos de alguna fuente normalmente proveniente del usuario lo que puede hacer que sean vulnerables a inyección sql si no se sanean las entradas como por ejemplo



eso es un ejemplo de una vulnerabilidad grave en la seguridad de una aplicación web o no porque si el usuario introdujese un valor como  nos encontraríamos con que la sentencia ejecutada sería



y se eliminaría la tabla usuarios con todos los datos contenidos en ella 

cómo puedo evitar que la inyección sql ocurra en php



In [19]:
#Retirando espaços duplicados
def substitui_por_espaco(textos,regex):
    if type(textos) == str:
        texto_sem_espaco = regex.sub("  ",textos)
        return texto_sem_espaco
    else:
        return [regex.sub(" ",texto) for texto in textos]
regex_espaco = re.compile(r"( )+")
regex_linha = re.compile(r"(\n)")
regex_digito = re.compile(r"\d")
dados_es_tratado_v2 = substitui_por_espaco(dados_es_tratado,regex_linha)
substitui_por_espaco(dados_es_tratado_v2,regex_espaco)

'Las  sentencias  dinámicas  son  sentencias  SQL  que  se  crean  como  cadenas  de  texto  strings  y  en  las  que  se  insertanconcatenan  valores  obtenidos  de  alguna  fuente  normalmente  proveniente  del  usuario  lo  que  puede  hacer  que  sean  vulnerables  a  inyección  SQL  si  no  se  sanean  las  entradas  como  por  ejemplo  Eso  es  un  ejemplo  de  una  vulnerabilidad  grave  en  la  seguridad  de  una  aplicación  web  o  no  porque  si  el  usuario  introdujese  un  valor  como  nos  encontraríamos  con  que  la  sentencia  ejecutada  sería  Y  se  eliminaría  la  tabla  Usuarios  con  todos  los  datos  contenidos  en  ella  Cómo  puedo  evitar  que  la  inyección  SQL  ocurra  en  PHP  '

In [20]:
dados_es_tratado_v2

'Las sentencias dinámicas son sentencias SQL que se crean como cadenas de texto strings y en las que se insertanconcatenan valores obtenidos de alguna fuente normalmente proveniente del usuario lo que puede hacer que sean vulnerables a inyección SQL si no se sanean las entradas como por ejemplo        Eso es un ejemplo de una vulnerabilidad grave en la seguridad de una aplicación web o no porque si el usuario introdujese un valor como  nos encontraríamos con que la sentencia ejecutada sería        Y se eliminaría la tabla Usuarios con todos los datos contenidos en ella     Cómo puedo evitar que la inyección SQL ocurra en PHP  '

In [21]:
#Questões de português pós remoções de pontuação, quebra de linha, e espaço duplicado, além de padronização em minúsculo
quest_port_nopunct = remove_texto(dados_portugues.Pos_tratamento,regex_pontuacao)
quest_port_minusculo = minusculo(quest_port_nopunct)
quest_port_noline = substitui_por_espaco(quest_port_minusculo,regex_linha)
quest_port_nospace= substitui_por_espaco(quest_port_noline,regex_espaco)
quest_port_nodigit = remove_texto(quest_port_nospace,regex_digito)
dados_portugues["tratados_v2"] = quest_port_nodigit
quest_port_nodigit[0]

'se eu fizer o hash de senhas antes de armazenálas em meu banco de dados é suficiente para evitar que elas sejam recuperadas por alguém estou falando apenas da recuperação diretamente do banco de dados e não qualquer outro tipo de ataque como força bruta na página de login da aplicação keylogger no cliente e criptoanálise rubberhose qualquer forma de hash não vai impedir esses ataques tenho preocupação em dificultar ou até impossibilitar a obtenção das senhas originais caso o banco de dados seja comprometido como dar maior garantia de segurança neste aspecto quais preocupações adicionais evitariam o acesso às senhas existem formas melhores de fazer esse hash '

In [22]:
#Questões de espanhol pós remoções de pontuação, quebra de linha, e espaço duplicado, além de padronização em minúsculo
quest_es_nopunct = remove_texto(dados_espanhol.Pos_tratamento,regex_pontuacao)
quest_es_minusculo = minusculo(quest_es_nopunct)
quest_es_noline = substitui_por_espaco(quest_es_minusculo,regex_linha)
quest_es_nospace = substitui_por_espaco(quest_es_noline,regex_espaco)
quest_es_nodigit = remove_texto(quest_es_nospace,regex_digito)
dados_espanhol["tratados_v2"] = quest_es_nodigit
quest_es_nodigit[0]

'las sentencias dinámicas son sentencias sql que se crean como cadenas de texto strings y en las que se insertanconcatenan valores obtenidos de alguna fuente normalmente proveniente del usuario lo que puede hacer que sean vulnerables a inyección sql si no se sanean las entradas como por ejemplo eso es un ejemplo de una vulnerabilidad grave en la seguridad de una aplicación web o no porque si el usuario introdujese un valor como nos encontraríamos con que la sentencia ejecutada sería y se eliminaría la tabla usuarios con todos los datos contenidos en ella cómo puedo evitar que la inyección sql ocurra en php '

In [23]:
#Questões de inglês pós remoções de pontuação, quebra de linha, e espaço duplicado, além de padronização em minúsculo
quest_eng_nopunct = remove_texto(dados_ingles.Pos_tratamento,regex_pontuacao)
quest_eng_minusculo = minusculo(quest_eng_nopunct)
quest_eng_noline = substitui_por_espaco(quest_eng_minusculo,regex_linha)
quest_eng_nospace = substitui_por_espaco(quest_eng_noline,regex_espaco)
quest_eng_nodigit = remove_texto(quest_eng_nospace,regex_digito)
dados_ingles["tratados_v2"] = quest_eng_nodigit
quest_eng_nodigit[0]

'here is a piece of c code that seems very peculiar for some strange reason sorting the data miraculously makes the code almost six times faster without the code runs in  seconds with the sorted data the code runs in  seconds initially i thought this might be just a language or compiler anomaly so i tried it in java with a somewhat similar but less extreme result my first thought was that sorting brings the data into the cache but then i thought how silly that is because the array was just generated what is going on why is it faster to process a sorted array than an unsorted array the code is summing up some independent terms and the order should not matter '

In [24]:
#Utilizando bigramas para modelo de linguagem
texto_bigrama = "alura"
bigramas_texto = bigrams(texto_bigrama)
print(list(bigramas_texto))

[('a', 'l'), ('l', 'u'), ('u', 'r'), ('r', 'a')]


In [25]:
#Incluindo fakechar para manter as proporções de probabilidade do último e primeiro caractere equilibradas
print(list(bigrams(pad_both_ends(texto_bigrama,n=2))))

[('<s>', 'a'), ('a', 'l'), ('l', 'u'), ('u', 'r'), ('r', 'a'), ('a', '</s>')]


In [26]:
dados_portugues["idioma"] = "port"
dados_espanhol["idioma"] = "esp"
dados_ingles["idioma"] = "ing"

In [27]:
port_treino,port_test = train_test_split(dados_portugues.tratados_v2,test_size=0.2,random_state=42)
port_treino

249    estou trabalhando em um sistema muito grande u...
433    estou estudando rest e desenvolvimento de web ...
19     como saber se o código é de fácil leitura e ma...
322    um heisenbug é um bug que muda seu comportamen...
332    é mais vantajoso utilizar um tipo ansi ao invé...
                             ...                        
106    qual a diferença entre ienumerable iqueryable ...
270    eu estudei java há bastante tempo e estou bem ...
348    é comum um usuário não poder usar todos os ele...
435    para um único script a ordem de cargaexecução ...
102    é muito comum falarse sobre o padrão mvc model...
Name: tratados_v2, Length: 400, dtype: object

In [38]:
ing_treino,ing_test = train_test_split(dados_ingles.tratados_v2,test_size=0.2,random_state=42)
ing_treino

249    we are working on an s version and this platfo...
433    ive been using git for a while now and have re...
19      can i use comments inside a json file if so how 
322    how do i determine if variable is or my code i...
332    what is this idiom and when should it be used ...
                             ...                        
106     possible duplicate how to efficiently iterate...
270    how can i display json in an easytoread for hu...
348    is there a way to convert a string from upperc...
435    i am working on a web application where i want...
102    i recently forked a project and applied severa...
Name: tratados_v2, Length: 400, dtype: object

In [39]:
esp_treino,esp_test = train_test_split(dados_espanhol.tratados_v2,test_size=0.2,random_state=42)
esp_treino

249    si tengo por ejemplo varios y le hago click a ...
433    he agregado una ordenación de noticias por sus...
19     recientemente mi sitio fue hackeado los atacan...
322    pongamos que tengo un array bidimensional de e...
332    al sacar los valores de una encontré que puedo...
                             ...                        
106    un compañero de trabajo creó un llamado quiero...
270    tengo que procesar una gran cantidad de frases...
348    uso un userscript para cambiar ciertas funcion...
435    alguno me puede explicar cómo funciona el oper...
102    quiero mostrar el teclado virtual softkeyboard...
Name: tratados_v2, Length: 400, dtype: object

In [32]:
todas_as_palavras = " ".join(port_treino)
port_tokenizada = nltk.tokenize.WhitespaceTokenizer().tokenize(todas_as_palavras)
port_treino_bigram,vocab = padded_everygram_pipeline(2,port_tokenizada)

In [33]:
#Modelo
modelo_port = MLE(2)
modelo_port.fit(port_treino_bigram,vocab)

In [34]:
#Gerando palavras(caractere por caractere) com o modelo treinado com dados em português e cuja inicial da palavra sejas m
modelo_port.generate(4,"m",random_seed=42)

['e', '</s>', '<s>', 'd']

In [35]:
#Frequência de quais caracteres são os primeiros a seguir o caractere m
modelo_port.counts[["m"]].items()

dict_items([('</s>', 2693), ('a', 2096), ('u', 228), ('v', 23), ('p', 682), ('e', 1469), ('o', 854), ('s', 12), ('b', 147), ('i', 300), ('í', 16), ('g', 4), ('é', 70), ('á', 43), ('y', 14), ('ó', 39), ('l', 22), ('m', 26), ('ã', 3), ('â', 10), ('d', 1), ('ú', 2), ('w', 1), ('f', 1), ('q', 5)])

In [44]:
#Calculando a perplexidade(medida de evaluation do modelo) 
frase_teste = "bom dia"
frase_teste_tokenizada = nltk.tokenize.WhitespaceTokenizer().tokenize(frase_teste)
frase_teste_fakechar = [list(pad_both_ends(texto,2)) for texto in frase_teste_tokenizada]
frase_teste_bigrama = [list(bigrams(texto)) for texto in frase_teste_fakechar]
frase_teste_bigrama

[[('<s>', 'b'), ('b', 'o'), ('o', 'm'), ('m', '</s>')],
 [('<s>', 'd'), ('d', 'i'), ('i', 'a'), ('a', '</s>')]]

In [167]:
modelo_port.perplexity(frase_teste_bigrama[0])

7.867530654448737

In [42]:
#Definindo função para treinar o modelo
def treina_modelo_mle(lista_texto):
    todas_as_palavras = " ".join(lista_texto)
    palavras_tokenizadas = nltk.tokenize.WhitespaceTokenizer().tokenize(todas_as_palavras)
    bigram, vocab = padded_everygram_pipeline(2,palavras_tokenizadas)
    modelo = MLE(2)
    modelo.fit(bigram,vocab)
    return modelo

In [63]:
#Definindo função para calcular perplexidade
def calcula_perplexidade(texto,modelo):
    texto_tokenizado = nltk.tokenize.WhitespaceTokenizer().tokenize(texto)
    texto_fakechar = [list(pad_both_ends(texto,2)) for texto in texto_tokenizado]
    texto_bigram = [list(bigrams(texto)) for texto in texto_fakechar]
    resultado = [modelo.perplexity(texto) for texto in texto_bigram]
    return sum(resultado)

In [45]:
calcula_perplexidade(frase_teste,modelo_port)

22.70732882059757

In [51]:
modelo_ing = treina_modelo_mle(ing_treino)
modelo_esp = treina_modelo_mle(esp_treino)
modelo_port = treina_modelo_mle(port_treino)

In [84]:
calcula_perplexidade(port_test.iloc[0],modelo_port)

inf

In [75]:
calcula_perplexidade(ing_test.iloc[3],modelo_ing)

465.7445725482562

In [72]:
ing_test.iloc[1]

'yesterday i was pairing the socks from the clean laundry and figured out the way i was doing it is not very efficient i was doing a naive searchnbsp picking one sock and iterating the pile in order to find its pair this requires iterating over n n n socks on average as a computer scientist i was thinking what i could do sorting according to sizecolor of course came to mind to achieve an onlogn solution hashing or other notinplace solutions are not an option because i am not able to duplicate my socks though it could be nice if i could so the question is basically given a pile of pairs of socks containing elements assume each sock has exactly one matching pair what is the best way to pair them up efficiently with up to logarithmic extra space i believe i can remember that amount of info if needed i will appreciate an answer that addresses the following aspects a general theoretical solution for a huge number of socks the actual number of socks is not that large i dont believe my spouse

In [85]:
port_test.iloc[0]

'quais as principais diferenças entre as engines e e qual a situação mais apropriada para o uso de cada sei que não suporta mas suporta enquanto no é o contrário pelo menos até a versão  do mysql por que um sgbdr rdbms tem como padrão uma engine que não oferece meios para relacionar as tabelas '

In [88]:
#Modelo de estimativa de Likelihood suavizado pela transformada de Laplace para evitar a perplexidade ser igual a infinito
def treina_modelo_Laplace(lista_texto):
    todas_as_palavras = " ".join(lista_texto)
    palavras_tokenizadas = nltk.tokenize.WhitespaceTokenizer().tokenize(todas_as_palavras)
    bigram, vocab = padded_everygram_pipeline(2,palavras_tokenizadas)
    modelo = Laplace(2)
    modelo.fit(bigram,vocab)
    return modelo

In [90]:
port_laplace = treina_modelo_Laplace(port_treino)
calcula_perplexidade(port_test.iloc[0],port_laplace)

556.1304349437955

In [91]:
ing_laplace = treina_modelo_Laplace(ing_treino)
calcula_perplexidade(ing_test.iloc[0],ing_laplace)

180.42718559493858

In [93]:
esp_laplace = treina_modelo_Laplace(esp_treino)
calcula_perplexidade(esp_test.iloc[0],esp_laplace)

405.7119034147504

In [94]:
def atribui_idioma(lista_texto):
    idioma = []
    for i in lista_texto:
        perplexidade_pt = calcula_perplexidade(i,port_laplace)
        perplexidade_en = calcula_perplexidade(i,ing_laplace)
        perplexidade_esp = calcula_perplexidade(i,esp_laplace)
        if perplexidade_pt < perplexidade_en and perplexidade_pt < perplexidade_esp:
            idioma.append("portugues")
        elif perplexidade_en < perplexidade_pt and perplexidade_en < perplexidade_esp:
            idioma.append("ingles")
        else:
            idioma.append("espanhol")
    return idioma


In [103]:
#Calculando a taxa de acerto do tradutor com os splits de teste de cada idioma
taxa_pt = (atribui_idioma(port_test).count("portugues"))/len(atribui_idioma(port_test))
print(f"A taxa de acerto para o português é de {taxa_pt}")
taxa_en = (atribui_idioma(ing_test).count("ingles"))/len(atribui_idioma(ing_test))
print(f"A taxa de acerto para o inglês é de {taxa_en}")
taxa_esp = (atribui_idioma(esp_test).count("espanhol"))/len(atribui_idioma(esp_test))
print(f"A taxa de acerto para o espanhol é de {taxa_esp}")

A taxa de acerto para o português é de 1.0
A taxa de acerto para o inglês é de 1.0
A taxa de acerto para o espanhol é de 0.97
